In [2]:
%load_ext autoreload
%autoreload 2

import asyncio
import pandas as pd
from vector_search import VectorSearch, build_context_string
from vexa import VexaAPI
import pandas as pd

In [19]:

# Initialize the VexaAPI
vexa = VexaAPI(token='0214d19adcc241b385dad8ddccbfd17a')
vexa = VexaAPI()
from core import generic_call, system_msg, user_msg
from pydantic_models import Summary

User information retrieved successfully.
User information retrieved successfully.


In [20]:


def create_exploded_points_df(output):
    points_data = output[0].model_dump()['points']
    df_points = pd.DataFrame(points_data)
    df_points = df_points.rename(columns={'c': 'point'})
    def create_range(start, end):
        return list(range(int(start), int(end) + 1))
    df_points['range'] = df_points.apply(lambda row: create_range(row['s'], row['e']), axis=1)
    df_exploded = df_points.explode('range')
    df_exploded = df_exploded.drop(columns=['s', 'e'])
    df_exploded = df_exploded.reset_index(drop=True)
    return df_exploded

In [21]:
def combine_initials_and_content(group):
    combined = group['speaker'].fillna('').astype(str) + ': ' + group['content'].fillna('')


    return ' '.join(combined)

In [22]:
from prompts import Prompts
prompts = Prompts()

In [27]:
analyzer = VectorSearch(gpu_device=3)
meetings = vexa.get_meetings()
meetings = meetings[-200:]


In [25]:
analyzer.collection_name
analyzer.delete_collection()

In [28]:
analyzer.collection_name

'transcript_collection'

In [29]:
len(meetings)

200

In [30]:
vexa.user_id

'ef7c085b-fdb5-4c94-b7b6-a61a3d04c210'

In [32]:
vexa.user_name

'Dmitriy Grankin'

In [33]:
#analyzer.delete_collection()

In [34]:
for meeting in reversed(meetings):
    if meeting['finish_timestamp']:
        meeting_id = meeting['id']
        if not analyzer.check_meeting_session_id_exists(meeting_id):
            result = vexa.get_transcription(meeting_session_id=meeting_id,use_index=True)
            if result:
                df, formatted_input, start_datetime, speakers = result
                output = await Summary.call([system_msg(prompts.think+ prompts.summarize_meeting+ '.The User: '+ vexa.user_name),user_msg(formatted_input)],model='gpt-4o-mini')
                df_exploded = create_exploded_points_df(output)
                joined_df = df_exploded.join(df, on='range', rsuffix='_transcript')

                points_with_qoutes = joined_df.groupby('point').apply(combine_initials_and_content)
                points_with_qoutes.name = 'qoutes'
                points_with_qoutes = points_with_qoutes.reset_index().to_dict(orient='records')
                summary = output[0].summary
                meeting_name = output[0].meeting_name
                
                if points_with_qoutes:
                    chunks = [f"{summary}\n\n{p['point']}\n\n{p['qoutes']}" for p in points_with_qoutes]
                    points = [p['point'] for p in points_with_qoutes]
                    qoutes = [p['qoutes'] for p in points_with_qoutes]
                    await analyzer.add_summary(meeting_name, summary, start_datetime, speakers, meeting_id,vexa.user_id,vexa.user_name)
                    await analyzer.update_vectorstore_with_qoutes(chunks, points, qoutes, start_datetime, speakers, meeting_id,vexa.user_id,vexa.user_name)


/tmp/ipykernel_168103/2454266488.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  points_with_qoutes = joined_df.groupby('point').apply(combine_initials_and_content)
/tmp/ipykernel_168103/2454266488.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  points_with_qoutes = joined_df.groupby('point').apply(combine_initials_and_content)
/tmp/ipykernel_168103/2454266488.py:12: DeprecationWarning: DataFrameG